In [1]:
from sklearn.datasets import fetch_20newsgroups
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from collections import defaultdict
import gensim
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
     
def pre_processData(newsgroups_train):
    tokenizer = RegexpTokenizer(r'\w+')
    for i in range(len(newsgroups_train)):
        newsgroups_train[i] = newsgroups_train[i].lower()
        newsgroups_train[i] = tokenizer.tokenize(newsgroups_train[i])
    newsgroups_train = [[token for token in doc if not token.isdigit()] for doc in newsgroups_train]
    #newsgroups_train = [[token for token in doc if len(token) > 3] for doc in newsgroups_train]
    lemmatizer = WordNetLemmatizer()
    newsgroups_train = [[lemmatizer.lemmatize(token) for token in doc] for doc in newsgroups_train]
    return newsgroups_train


def remove_stopwords(documents,news_labels):
    temp_corpus = {}
    temp_label = {}
    stop_en = stopwords.words('english')
    i = 0
    for index, words in enumerate(documents):
        rwords=[]
        for word in words:
            if word not in stop_en:
                rwords.append(word)
        if rwords:
            temp_corpus[i] = rwords
            temp_label[i] = news_labels[index]
            i = i+1
    return temp_corpus,temp_label

def process_wordvectors(vocab,vectors,documents,news_labels):
    useable_vocab = 0
    unusable_vocab = 0
    temp_corpus = {}
    temp_labels = {}
    temp_unvecs = {}
    i=0
    for index in range(len(documents)):
        filter_word = []
        words = documents[index]
        temp_unvecs[i] = {}
        for word in words:
            try:
                vectors[word]
                vocab.add(word)
                filter_word.append(word)
                useable_vocab += 1
            except:
                unusable_vocab += 1
                if word in temp_unvecs[i].keys():
                    temp_unvecs[i][word] +=1
                else:
                     temp_unvecs[i][word] = 1
                continue
        if filter_word and len(words)>0 :
            temp_corpus[i] = filter_word
            temp_labels[i] = news_labels[index]
            #print("doc ",i," not vec word len ",temp_unvecs[i])
            i= i+1
    print("There are {0} words that could be convereted to word vectors in your corpus \n" \
          "There are {1} words that could NOT be converted to word vectors".format(useable_vocab, unusable_vocab))
    print("doc num: ",i)
    print("label num: ",i)
    return temp_corpus,temp_labels,temp_unvecs
import json
if __name__ == "__main__":
    wordvec_fileapth = "glove.6B.50d.txt"
    #remove = ('headers', 'footers', 'quotes')
    #fectch_corpus = fetch_20newsgroups(subset='train', remove=remove)
    #newsgroups_train = fectch_corpus.data
    #print("target_names len ",len(fectch_corpus.target_names))
    #print("newsgroups_train size ",len(newsgroups_train))
    #print("newsgroups_train type ",type(newsgroups_train) )
    newsgroups_train = list()
    news_labels = list()
    supwords_train = list()
    file_path="part_news2.txt"
    #file_path="C:/Users/aa6602623/PycharmProjects/GaussianLDA/News.txt"
    with open(file_path) as fp:
        lines = fp.read().split("\n")
        for line in lines:
            if line:
                text = json.loads(line)["textCleaned"].strip()
                label = json.loads(line)["clusterNo"]
                newsgroups_train.append(text)
                news_labels.append(label)
    fp.close()
    corpus = pre_processData(newsgroups_train)
    corpus,news_labels = remove_stopwords(corpus,news_labels)
    vectors = gensim.models.KeyedVectors.load_word2vec_format(fname=wordvec_fileapth, binary=False)
    vocab = set([])
    corpus,news_labels,supwords_train =  process_wordvectors(vocab,vectors,corpus,news_labels)
    print(len(news_labels))
    print(len(corpus))

There are 68070 words that could be convereted to word vectors in your corpus 
There are 689 words that could NOT be converted to word vectors
doc num:  11108
label num:  11108
11108
11108


In [6]:
import timeit
from scipy.special import gamma, gammaln, loggamma
from numpy import log, pi, linalg, exp, e
import random
from scipy import stats
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)
import sys
doc_words = {}
for docID in range(len(corpus)):
    doc_words[docID] = {}
    words = corpus[docID]
    for word in words:
        if word not in doc_words[docID].keys():
            doc_words[docID][word] = 0
        doc_words[docID][word] += 1
        
alpha = 0.03
beta = 0.03
docID_assign_z = {}
m_z = {}
n_z = {}
n_w = {}
Topics = []
docID = 0
initial_z = 0
V = set()
D = set()
beta_topic_sum = {}
beta_topic_v = {}
gamma = 0.03
gammaS = 0.0000001

D.add(docID)
docID_assign_z[docID] = initial_z
words = corpus[docID]
if initial_z not in m_z.keys():
    m_z[initial_z] = set()
m_z[initial_z].add(docID)
for word in words:
    if initial_z not in n_w.keys():
        n_w[initial_z] = 0
    if initial_z not in n_z.keys():
        n_z[initial_z] = {}
    if word not in n_z[initial_z].keys():
        n_z[initial_z][word] = 0
    n_z[initial_z][word] += 1
    n_w[initial_z] += 1
    V.add(word)
def sum_topic_word():
    global_important_word = []
    for k in topic_keyword.keys():
        global_important_word.extend(list(topic_keyword[k]))
    x = None
    if len(global_important_word) !=0:
        vec_dim = 0
        for word in global_important_word:
            vec_dim += 1
            if x is not None:
                x = np.row_stack((x, vectors[word]))
            else:
                x = vectors[word] 
        global_v_bar_k[0] = None
        if  vec_dim > 1:
            global_v_bar_k[0]= np.mean(x,axis = 0)[:,None]
        else:
            x = x[:,None]
            global_v_bar_k[0] = x
        x = x.T
        global_kappa_k[0] =  kappa0 + vec_dim
        global_N[0] = vec_dim
        global_C_k[0] = (x - global_v_bar_k[0]).dot((x - global_v_bar_k[0]).T)
        global_mu_k[0] = (kappa0 * mu0 + global_N[0] * global_v_bar_k[0]) / global_kappa_k[0]
        global_psi_k[0] = psi + global_C_k[0] + (kappa0*global_N[0] / global_kappa_k[0])* ( (global_v_bar_k[0] - mu0).T.dot(global_v_bar_k[0] - mu0) )
        global_nu_k[0] = nu0 + vec_dim
        shakage_v = vec_dim + 1
        global_cov_k[0] = global_psi_k[0] / (shakage_v) 
        global_inv_cov_k[0] = np.linalg.inv(global_cov_k[0])
        #global_cov_det_k[0] = np.linalg.det(global_cov_k[0])
    else:
        shakage_v = 20
        global_cov_k[0] = psi/ (shakage_v) 
        global_mu_k[0] = mu0
        global_inv_cov_k[0] = np.linalg.inv(global_cov_k[0])
        #global_cov_det_k[0] = np.linalg.det(global_cov_k[0])
    return global_important_word
def sampleBetaAssignment(k, word,iter,total_iter, max_word_prob):
    if beta_topic_v[k][word] == 1:
        beta_topic_sum[k] -= 1
    pBetaAllOthers = beta_topic_sum[k]
    
    log_true = (n_z[k][word] / n_w[k]) / max_word_prob
    log_false = 1 - log_true
    
    log_p = []
    if log_false <0 or log_true < 0:
        a = 1/0
    log_p.append(log_false)
    log_p.append(log_true)
    
    sum_pro=sum(log_p)
    normalized_posterior = [i/sum_pro for i in log_p]    
    update_k = np.random.choice( 2 , 1, p=normalized_posterior)[0]
    if iter ==  total_iter - 1:
        update_k = 0
        if log_false < log_true:
            update_k = 1
    if update_k == 1:
        beta_topic_v[k][word] = 1
        beta_topic_sum[k] += 1
        topic_keyword[k].add(word)
    else:
        beta_topic_v[k][word] = 0
        topic_keyword[k].discard(word)

In [7]:
compara_batch = [len(corpus)]

In [11]:
nmi_compara_batch = []
mean_each_batch_nmi = {}
iteration_nmi = []
iteration_coherence = {}
iteration_topics = {}
iteration_NMI = {}
for i_batch in compara_batch:
    start = 0
    end = 0
    total_batch = None
    if len(corpus) % i_batch == 0:
        total_batch = int(len(corpus) / i_batch)
    else:
        total_batch = int(len(corpus) / i_batch) +1
    mean_each_batch_nmi[i_batch] = []
    for batch in range(total_batch):
        ave_nmi = []
        for mean_i in range(1):
        
            docID_assign_z = {}
            m_z = {}
            n_z = {}
            n_w = {}
            Topics = []
            V = set()
            D = set()
            beta_topic_v = {}
            beta_topic_sum = {}
            topic_keyword = {}
            
            global_v_bar_k = {}
            global_C_k= {}
            global_mu_k= {}
            global_psi_k= {}
            global_nu_k= {}
            global_kappa_k= {}
            global_cov_k= {}
            global_N= {}
            global_inv_cov_k= {}
            global_cov_det_k= {}
            kappa0 = 0.01
            dim = 50
            vec_x = 1.0
            nu0 = dim
            psi = np.eye(dim)
            mu0 = np.array([vec_x for i in range(dim)])[:,None]
            
            alpha = 0.03
            #alpha = 0.003
            
            gamma = 30
            gammaS = 0.03
            #gamma = 0.03
            #gammaS = 0.0000001
            global_word = {}
            global_important_word = None
            
            end = i_batch * (batch + 1)
            if end > len(corpus):
                end = len(corpus)
            total_iter = 10
            for iter in range(total_iter):
                iteration_coherence[iter] = {}
                iteration_topics[iter] = {}
                iteration_NMI[iter] = {}
                print("iter ",iter, " total K ",len(Topics))
#                 for k in Topics:
#                     k_words = set()
#                     for word in beta_topic_v[k]:
#                         if beta_topic_v[k][word] == 1:
#                             k_words.add(word)
                    #print("k ",k , "beta_topic_sum ",beta_topic_sum[k])
                    #print("k ",k , "n_w ",n_w[k])
                #print()
                for docID in range(start,end):
                    if docID%100 == 0:
                        print("iter ",iter, " doc ",docID)
                        print("total K ",len(Topics))
#                     if docID%5000 == 0:
#                         #print("iter ",iter, " m_z ",m_z)
#                         print("iter ",iter, " total K ",Topics)
#                         print()
                    words = corpus[docID]
                    D.discard(docID)
                    if docID in docID_assign_z.keys():
                        before_k = docID_assign_z[docID]
                        m_z[before_k].discard(docID)
                        for word in words:
                            global_word[word] -= 1
                            n_z[before_k][word] -= 1
                            n_w[before_k] -=1
#                         for word in n_z[before_k].keys():
#                             if n_z[before_k][word] > 0:
#                                 sampleBetaAssignment(before_k,word)
                        k = before_k
                        max_word_prob = n_w[k]
                        if max_word_prob != 0:
                            max_word_prob = max(n_z[k].values()) / max_word_prob

                        for word in n_z[k].keys():
                                if word not in beta_topic_v[k].keys():
                                        beta_topic_v[k][word] = 0
                                if n_z[k][word] > 0: 
                                    sampleBetaAssignment(k,word,iter,total_iter,max_word_prob)
                                else:
                                    if beta_topic_v[k][word] == 1:
                                        beta_topic_sum[k] -= 1
                                        beta_topic_v[k][word] = 0  
                                        topic_keyword[k].discard(word)
                        if docID % 500 == 0:
                            global_important_word = sum_topic_word()
                    else:
                        before_k = -1
                        
                    if len(D) == 0 and len(V) == 0:
                        choose_k = 0
                        D.add(docID)
                        docID_assign_z[docID] = choose_k
                        if choose_k not in beta_topic_v.keys(): 
                            beta_topic_v[choose_k] = {}
                        if choose_k not in beta_topic_sum.keys():
                            beta_topic_sum[choose_k] = 0
                        if choose_k not in m_z.keys():
                            m_z[choose_k] = set()
                        if choose_k not in topic_keyword.keys():
                            topic_keyword[choose_k] = set()
                        m_z[choose_k].add(docID)
                        for word in words:
                            if choose_k not in n_w.keys():
                                n_w[choose_k] = 0
                            if choose_k not in n_z.keys():
                                n_z[choose_k] = {}
                            if word not in n_z[choose_k].keys():
                                n_z[choose_k][word] = 0
                            if word not in beta_topic_v[choose_k].keys():
                                beta_topic_v[choose_k][word] = 0
                            if word not in global_word.keys():
                                global_word[word] = 0
                            global_word[word] += 1
                            n_z[choose_k][word] += 1
                            n_w[choose_k] += 1
                            V.add(word)
                        if choose_k not in Topics:
                            Topics.append(choose_k)
#                         if choose_k == Topics:
#                             Topics += 1  
                    else:
                        log_pro = []
                        
                        must_update_flag = 0
                        update_pro = 1
                        not_update_pro = 1
                        if_update_k = []
                        for word in words:
                            if word in global_important_word:
                                must_update_flag = 1
                                break
                            x = vectors[word][:,None]
                            LLcomp = (x - global_mu_k[0]).T.dot(global_inv_cov_k[0]).dot((x - global_mu_k[0])) 
                            pro = stats.chisqprob(LLcomp, dim)[0][0]
                            update_pro *= pro
                            not_update_pro *= (1 - pro)
                            
                        if must_update_flag == 0:
                            if_update_k.append(not_update_pro)
                            if_update_k.append(update_pro)
                            sum_pro=sum(if_update_k)
                            normalized_posterior = [i/sum_pro for i in if_update_k]    
                            update_k = np.random.choice( 2 , 1, p=normalized_posterior)[0]
                        else:
                            update_k = 1
                        
                        choose_k = None
                        if update_k == 1:
                            for k in Topics:
                                pro_k = len(m_z[k])
                                if pro_k == 0:
                                    log_pro.append(0)
                                else:
                                    i = 0
                                    for word in words:
                                        if word not in n_z[k].keys():
                                            n_z[k][word] = 0
                                        bias_flag = 0
                                        if word in beta_topic_v[k].keys():
                                            bias_flag = beta_topic_v[k][word]
                                        for j in range(doc_words[docID][word]):
                                            #pro_k *= (n_z[k][word]+ beta + j) / (n_w[k] + len(V)*beta + i) 
                                            pro_k *= (n_z[k][word]+ bias_flag*gamma + gammaS +j)/(n_w[k]+beta_topic_sum[k]*gamma+len(V)*gammaS +i) 
                                            i += 1
                                    log_pro.append(pro_k)
                            sum_pro=sum(log_pro)
                            normalized_posterior = [i/sum_pro for i in log_pro]  
                            select_k = None
                            if iter == (total_iter - 1):
                                select_k = normalized_posterior.index(max(normalized_posterior))

                            else:
                                select_k = np.random.choice( len(Topics) , 1, p=normalized_posterior)[0]   
                            choose_k = Topics[select_k]
                        else:
                            choose_k = np.max(Topics) + 1
                       
                            
                        D.add(docID)
                        docID_assign_z[docID] = choose_k
                        if choose_k not in m_z.keys():
                            m_z[choose_k] = set()
                        m_z[choose_k].add(docID)
                        if choose_k not in beta_topic_v.keys(): 
                            beta_topic_v[choose_k] = {}
                        if choose_k not in beta_topic_sum.keys():
                            beta_topic_sum[choose_k] = 0
                        if choose_k not in topic_keyword.keys():
                            topic_keyword[choose_k] = set()
                        for word in words:
                            if choose_k not in n_w.keys():
                                n_w[choose_k] = 0
                            if choose_k not in n_z.keys():
                                n_z[choose_k] = {}
                            if word not in n_z[choose_k].keys():
                                n_z[choose_k][word] = 0
                            if word not in beta_topic_v[choose_k].keys():
                                beta_topic_v[choose_k][word] = 0
                            if word not in global_word.keys():
                                global_word[word] = 0
                            global_word[word] += 1
                            n_z[choose_k][word] += 1
                            n_w[choose_k] += 1
                            V.add(word)
                        if choose_k not in Topics:
                            Topics.append(choose_k)
                            
                    count_k = []
                    for k in Topics:
                        if k in m_z.keys() and len(m_z[k]) == 0:
                            m_z.pop(k, None)
                            n_z.pop(k, None)
                            n_w.pop(k, None)
                            beta_topic_v.pop(k, None)
                            beta_topic_sum.pop(k, None)
                            beta_topic_v.pop(k, None)
                            topic_keyword.pop(k, None)
                            count_k.append(k)
                    for k in count_k:
                        Topics.remove(k)   
                        
                    k = choose_k
                    max_word_prob = n_w[k]
                    if max_word_prob != 0:
                        max_word_prob = max(n_z[k].values()) / max_word_prob

                    for word in n_z[k].keys():
                            if word not in beta_topic_v[k].keys():
                                    beta_topic_v[k][word] = 0
                            if n_z[k][word] > 0: 
                                sampleBetaAssignment(k,word,iter,total_iter,max_word_prob)
                            else:
                                if beta_topic_v[k][word] == 1:
                                    beta_topic_sum[k] -= 1
                                    beta_topic_v[k][word] = 0   
                                    topic_keyword[k].discard(word)
                    if docID % 100 == 0:
                        global_important_word = sum_topic_word()                
#                 for k in Topics:
#                     max_word_prob = n_w[k]
#                     if max_word_prob != 0:
#                         max_word_prob = max(n_z[k].values()) / max_word_prob

#                     for word in n_z[k].keys():
#                             if word not in beta_topic_v[k].keys():
#                                     beta_topic_v[k][word] = 0
#                             if n_z[k][word] > 0: 
#                                 sampleBetaAssignment(k,word,iter,total_iter,max_word_prob)
#                             else:
#                                 if beta_topic_v[k][word] == 1:
#                                     beta_topic_sum[k] -= 1
#                                     beta_topic_v[k][word] = 0
                #spiltDocInTopic()
#                 for k in Topics:
#                     flag = 0
#                     print("k ", k)
#                     for word in beta_topic_v[k].keys():
#                         if beta_topic_v[k][word] == 1:
#                             flag = 1
#                             print("import word:",word)
#                     print()
                
                from sklearn.metrics.cluster import normalized_mutual_info_score
                nmi_sample = []
                nmi_result = []
                for key, value in news_labels.items():
                    if key < end and key >= start:
                        nmi_sample.append(value)
                        nmi_result.append(docID_assign_z[key])
                print("iter ",iter," ",normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result)))
                iteration_nmi.append(normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result)))
                
                threshold_loop = [0, 3, 5, 10, 15, 20]
                for threshold in threshold_loop:
                    count = 0
                    k_count = set()
                    for k in m_z.keys():
                        if len(m_z[k]) >= threshold:
                            count += 1
                            k_count.add(k)
                    import operator
                    post_topic_word = {}
                    for k in k_count:
                        post_topic_word[k] = {}
                        for word in V:
                            pro = None
                            if word in beta_topic_v[k] and beta_topic_v[k][word] == 1:
                                pro = (n_z[k][word]+ beta_topic_v[k][word]*gamma + gammaS) / (n_w[k] + beta_topic_sum[k]*gamma + len(V)*beta) 
                            elif word in n_z[k]:
                                pro = (n_z[k][word]+ gammaS) / (n_w[k] + beta_topic_sum[k]*gamma + len(V)*beta) 
                            else:
                                pro = (gammaS) / (n_w[k] + beta_topic_sum[k]*gamma + len(V)*beta) 
                            post_topic_word[k][word] = pro
                        post_topic_word[k] = sorted(post_topic_word[k].items(), key=operator.itemgetter(1), reverse=True)
                    top = 10
                    topic_coherence = []
                    for k in k_count:
                        top_words = []
                        pro = 0
                        for i in range(top):
                            top_words.append( post_topic_word[k][i][0] )

                        for i in range(len(top_words)-1):
                            for j in range(i+1,len(top_words) ):
                                count_i = 0
                                count_ij = 0
                                for docID in range( len(corpus) ):
                                    if top_words[i] in corpus[docID]:
                                        count_i += 1
                                        if top_words[j] in corpus[docID]:
                                            count_ij += 1
                                pro += log( (count_ij + 1) / count_i )
                        topic_coherence.append(pro)
                    iteration_coherence[iter][threshold] = np.mean(topic_coherence)
                    iteration_topics[iter][threshold] = count
                    
                    nmi_sample = []
                    nmi_result = []
                    for k in k_count:
                        for docID in m_z[k]:
                            nmi_sample.append(news_labels[docID])
                            nmi_result.append(docID_assign_z[docID])
                    iteration_NMI[iter][threshold] = normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result))
                    
                print("iter coherence: ",iteration_coherence)
                print("iter topics: ",iteration_topics)
                print("iter NMI: ",iteration_NMI)
                
                
            from sklearn.metrics.cluster import normalized_mutual_info_score
            nmi_sample = []
            nmi_result = []
            for key, value in news_labels.items():
                if key < end and key >= start:
                    nmi_sample.append(value)
                    nmi_result.append(docID_assign_z[key])
            ave_nmi.append(normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result)))
        mean_each_batch_nmi[i_batch].append(np.mean(ave_nmi))
        start = end 
        if batch%10 == 0:
            print("i_batch",i_batch , "batch ",batch)
            print(" ave_nmi ",np.mean(ave_nmi))
            print(" mean_each_batch_nmi ",np.mean(mean_each_batch_nmi[i_batch]) ,"max ", np.max(ave_nmi) ,"min ", np.min(ave_nmi) )
            print("total K ",len(Topics))
            print("global_important_word ",len(global_important_word))
            print("iteration_nmi ",iteration_nmi)
            print("iteration_coherence ",iteration_coherence)
            print()
    nmi_compara_batch.append(np.mean(mean_each_batch_nmi[i_batch]))
    print(compara_batch)
    print(nmi_compara_batch)

iter  0  total K  0
iter  0  doc  0
total K  0
iter  0  doc  100
total K  99
iter  0  doc  200
total K  112
iter  0  doc  300
total K  119
iter  0  doc  400
total K  123
iter  0  doc  500
total K  125
iter  0  doc  600
total K  126
iter  0  doc  700
total K  133
iter  0  doc  800
total K  134
iter  0  doc  900
total K  137
iter  0  doc  1000
total K  138
iter  0  doc  1100
total K  141
iter  0  doc  1200
total K  141
iter  0  doc  1300
total K  142
iter  0  doc  1400
total K  144
iter  0  doc  1500
total K  145
iter  0  doc  1600
total K  146
iter  0  doc  1700
total K  146
iter  0  doc  1800
total K  147
iter  0  doc  1900
total K  148
iter  0  doc  2000
total K  149
iter  0  doc  2100
total K  153
iter  0  doc  2200
total K  153
iter  0  doc  2300
total K  153
iter  0  doc  2400
total K  154
iter  0  doc  2500
total K  158
iter  0  doc  2600
total K  161
iter  0  doc  2700
total K  163
iter  0  doc  2800
total K  164
iter  0  doc  2900
total K  166
iter  0  doc  3000
total K  166
ite

iter  2  doc  100
total K  227
iter  2  doc  200
total K  226
iter  2  doc  300
total K  225
iter  2  doc  400
total K  224
iter  2  doc  500
total K  224
iter  2  doc  600
total K  224
iter  2  doc  700
total K  223
iter  2  doc  800
total K  224
iter  2  doc  900
total K  224
iter  2  doc  1000
total K  224
iter  2  doc  1100
total K  223
iter  2  doc  1200
total K  223
iter  2  doc  1300
total K  223
iter  2  doc  1400
total K  223
iter  2  doc  1500
total K  224
iter  2  doc  1600
total K  225
iter  2  doc  1700
total K  225
iter  2  doc  1800
total K  224
iter  2  doc  1900
total K  224
iter  2  doc  2000
total K  224
iter  2  doc  2100
total K  225
iter  2  doc  2200
total K  226
iter  2  doc  2300
total K  225
iter  2  doc  2400
total K  224
iter  2  doc  2500
total K  223
iter  2  doc  2600
total K  223
iter  2  doc  2700
total K  224
iter  2  doc  2800
total K  224
iter  2  doc  2900
total K  223
iter  2  doc  3000
total K  222
iter  2  doc  3100
total K  223
iter  2  doc  320

iter  3  doc  11000
total K  228
iter  3  doc  11100
total K  228
iter  3   0.8627709703224935
iter coherence:  {0: {0: -99.45237469627924, 3: -107.90463077573729, 5: -111.26447618648217, 10: -112.58037311288443, 15: -112.34069598570198, 20: -114.10974785146573}, 1: {0: -97.7502529291291, 3: -104.07349520000194, 5: -106.37165157450859, 10: -109.3457475201979, 15: -109.15124738157455, 20: -112.59101343808385}, 2: {0: -94.22221744563457, 3: -102.54322992435141, 5: -105.42764344628385, 10: -108.17886723345612, 15: -107.74326108597603, 20: -109.92813235805652}, 3: {0: -97.07362945585966, 3: -102.89574159609782, 5: -107.26440217788756, 10: -109.08071114843221, 15: -108.83985288496274, 20: -110.52253263906151}}
iter topics:  {0: {0: 225, 3: 168, 5: 148, 10: 131, 15: 118, 20: 103}, 1: {0: 226, 3: 164, 5: 148, 10: 125, 15: 117, 20: 103}, 2: {0: 224, 3: 163, 5: 145, 10: 127, 15: 119, 20: 103}, 3: {0: 228, 3: 172, 5: 147, 10: 122, 15: 115, 20: 106}}
iter NMI:  {0: {0: 0.8125584171202477, 3: 0.81

iter  5  doc  3800
total K  231
iter  5  doc  3900
total K  231
iter  5  doc  4000
total K  231
iter  5  doc  4100
total K  231
iter  5  doc  4200
total K  231
iter  5  doc  4300
total K  230
iter  5  doc  4400
total K  230
iter  5  doc  4500
total K  230
iter  5  doc  4600
total K  231
iter  5  doc  4700
total K  232
iter  5  doc  4800
total K  232
iter  5  doc  4900
total K  232
iter  5  doc  5000
total K  231
iter  5  doc  5100
total K  231
iter  5  doc  5200
total K  231
iter  5  doc  5300
total K  232
iter  5  doc  5400
total K  233
iter  5  doc  5500
total K  233
iter  5  doc  5600
total K  233
iter  5  doc  5700
total K  232
iter  5  doc  5800
total K  232
iter  5  doc  5900
total K  231
iter  5  doc  6000
total K  230
iter  5  doc  6100
total K  228
iter  5  doc  6200
total K  229
iter  5  doc  6300
total K  230
iter  5  doc  6400
total K  230
iter  5  doc  6500
total K  230
iter  5  doc  6600
total K  229
iter  5  doc  6700
total K  229
iter  5  doc  6800
total K  228
iter  5 

iter coherence:  {0: {0: -99.45237469627924, 3: -107.90463077573729, 5: -111.26447618648217, 10: -112.58037311288443, 15: -112.34069598570198, 20: -114.10974785146573}, 1: {0: -97.7502529291291, 3: -104.07349520000194, 5: -106.37165157450859, 10: -109.3457475201979, 15: -109.15124738157455, 20: -112.59101343808385}, 2: {0: -94.22221744563457, 3: -102.54322992435141, 5: -105.42764344628385, 10: -108.17886723345612, 15: -107.74326108597603, 20: -109.92813235805652}, 3: {0: -97.07362945585966, 3: -102.89574159609782, 5: -107.26440217788756, 10: -109.08071114843221, 15: -108.83985288496274, 20: -110.52253263906151}, 4: {0: -94.97479490676203, 3: -101.36360616724036, 5: -106.04216347473952, 10: -107.15622737031954, 15: -107.22341158650177, 20: -109.53120496154995}, 5: {0: -94.89616269074472, 3: -100.59794524209964, 5: -104.8906780699791, 10: -107.32379290367668, 15: -106.93536255696618, 20: -110.29462336424264}, 6: {0: -95.5401286795092, 3: -101.26612265173497, 5: -105.38264405615514, 10: -

iter  8  doc  100
total K  241
iter  8  doc  200
total K  241
iter  8  doc  300
total K  241
iter  8  doc  400
total K  242
iter  8  doc  500
total K  241
iter  8  doc  600
total K  240
iter  8  doc  700
total K  240
iter  8  doc  800
total K  240
iter  8  doc  900
total K  240
iter  8  doc  1000
total K  240
iter  8  doc  1100
total K  239
iter  8  doc  1200
total K  239
iter  8  doc  1300
total K  239
iter  8  doc  1400
total K  238
iter  8  doc  1500
total K  239
iter  8  doc  1600
total K  238
iter  8  doc  1700
total K  238
iter  8  doc  1800
total K  238
iter  8  doc  1900
total K  238
iter  8  doc  2000
total K  239
iter  8  doc  2100
total K  237
iter  8  doc  2200
total K  237
iter  8  doc  2300
total K  236
iter  8  doc  2400
total K  237
iter  8  doc  2500
total K  235
iter  8  doc  2600
total K  234
iter  8  doc  2700
total K  235
iter  8  doc  2800
total K  236
iter  8  doc  2900
total K  235
iter  8  doc  3000
total K  234
iter  8  doc  3100
total K  234
iter  8  doc  320

iter  9  doc  4400
total K  240
iter  9  doc  4500
total K  240
iter  9  doc  4600
total K  239
iter  9  doc  4700
total K  240
iter  9  doc  4800
total K  241
iter  9  doc  4900
total K  243
iter  9  doc  5000
total K  243
iter  9  doc  5100
total K  243
iter  9  doc  5200
total K  243
iter  9  doc  5300
total K  244
iter  9  doc  5400
total K  247
iter  9  doc  5500
total K  248
iter  9  doc  5600
total K  248
iter  9  doc  5700
total K  249
iter  9  doc  5800
total K  249
iter  9  doc  5900
total K  249
iter  9  doc  6000
total K  249
iter  9  doc  6100
total K  249
iter  9  doc  6200
total K  251
iter  9  doc  6300
total K  252
iter  9  doc  6400
total K  253
iter  9  doc  6500
total K  254
iter  9  doc  6600
total K  256
iter  9  doc  6700
total K  256
iter  9  doc  6800
total K  258
iter  9  doc  6900
total K  258
iter  9  doc  7000
total K  258
iter  9  doc  7100
total K  259
iter  9  doc  7200
total K  260
iter  9  doc  7300
total K  261
iter  9  doc  7400
total K  262
iter  9 

In [12]:
topic_word = {}
top_n = 10
for topic_num in n_z.keys():
    top_n_word = sorted(n_z[topic_num].keys(), key=lambda i: n_z[topic_num][i], reverse=True)[:top_n]
    topic_word[topic_num] = [i for i in top_n_word]

In [13]:
for i in topic_word.keys():
    print("Topic",topic_word[i][0],":",topic_word[i])

Topic nigella : ['nigella', 'lawson', 'drug', 'court', 'saatchi', 'charles', 'head', 'claim', 'cocaine', 'assistant']
Topic winning : ['winning', 'formula', 'york', 'knicks', 'searching', 'mourinho', 'double', 'player', 'jose', 'dribble']
Topic johansson : ['johansson', 'globe', 'scarlett', 'golden', 'ineligible', 'voice', 'performance', 'award', 'role', 'deemed']
Topic storm : ['storm', 'travel', 'thanksgiving', 'holiday', 'east', 'winter', 'weather', 'threatens', 'coast', 'rain']
Topic manning : ['manning', 'brady', 'peyton', 'tom', 'patriot', 'bronco', 'welker', 'wes', 'week', 'nfl']
Topic book : ['book', 'record', 'sell', 'psalm', 'auction', 'printed', 'expensive', 'fetch', 'bay', 'america']
Topic xbox : ['xbox', 'p', 'gamers', 'gift', 'guide', 'disappoint', 'box', 'review', 'black', 'big']
Topic launch : ['launch', 'philippine', 'campaign', 'program', 'nasa', 'commercial', 'crew', 'unicef', 'rihanna', 'help']
Topic nokia : ['nokia', 'lumia', 'phone', 'window', 'tablet', 'uk', 'lau